# Computation of source rate

In [ ]:
import h5py
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib.patches import Ellipse

# 1. Determine the spectrum of SPEs vs. gamma energy

### Read in the simulated PMT responses
The ROOT script `read_PMT.C` prints key information from the ROOT file output by Geant4. This can then be read in as a Pandas DataFrame and saved in a compressed .h5 file.

In [ ]:
initial_read = False
h5file = '/home/jrenner/local/hk/WCSim/WCSim_build/hits_NuPRISMBeamTest_mPMT.h5'

if(initial_read):
    
    # Read files containing the PMT coordinates and the simulated hits into Pandas dataframes.
    print("Reading files into dataframes...")
    df_pmt     = pd.read_csv('/home/jrenner/local/hk/WCSim/WCSim_build/NuPRISMBeamTest_mPMT_geom.dat', delimiter = " ")
    df_photons = pd.read_csv('/home/jrenner/local/hk/WCSim/WCSim_build/hits_NuPRISMBeamTest_mPMT.dat', delimiter = " ")

    # Output the dataframes to HDF5.
    print("Outputting files to HDF5...")
    df_pmt.to_hdf(h5file,key='pmts',complevel=9,complib='zlib')
    df_photons.to_hdf(h5file,key='photons',complevel=9,complib='zlib')

else:
    
    # Read the dataframes from HDF5.
    print("Reading the dataframes from HDF5...")
    df_pmt = pd.read_hdf(h5file,key='pmts')
    df_photons = pd.read_hdf(h5file,key='photons')

In [ ]:
# Show a few rows of the PMTs dataframe.
df_pmt.head()

In [ ]:
# Show a few rows of the photons dataframe.
df_photons.head()

In [ ]:
plt.hist(df_photons['energy'],bins=50)

### Processing for multi-energy gammas

In [ ]:
ewidth = 0.2
npmts = 2507
ebands = np.arange(0,10.0,ewidth)

avg_dict = {}
for emin in ebands:
    
    # Compute the emax for this band.
    emax = emin + ewidth
    
    # Set the values in the average dictionary.
    avg_dict.setdefault('emin',[])
    avg_dict.setdefault('emax',[])
    avg_dict.setdefault('nevts',[])
    
    # Set the emin and emax in the dictionary.
    avg_dict['emin'].append(emin)
    avg_dict['emax'].append(emax)
    
    # Select all events in this energy range.
    df = df_photons[(df_photons.energy > emin) & (df_photons.energy <= emax)]
    
    avg_dict['nevts'].append(len(df))
    
    # Compute the average number of photons per event for all PMTs.
    for npmt in range(npmts):
        
        pmt_str = 'pmt{}'.format(npmt+1)
        
        if(len(df) > 0):
            pmt_avg = df[pmt_str].mean()
        else:
            pmt_avg = 0.
            
        avg_dict.setdefault(pmt_str,[])
        avg_dict[pmt_str].append(pmt_avg)
        
# Make a new dataframe from the average dictionary.
df_avg = pd.DataFrame.from_dict(avg_dict)

In [ ]:
# Make the plot of average number of photons per gamma.
pmt_sums = df_avg.loc[:, (df_avg.columns != 'nevts') & (df_avg.columns != 'emin') & (df_avg.columns != 'emax')].sum()
pmt_max = pmt_sums.idxmin()

eplt = ((df_avg['emin'] + df_avg['emax'])/2).values
spes = df_avg[pmt_max].values
spes_err = np.nan_to_num(np.sqrt(spes/df_avg['nevts'].values))

In [ ]:
plt.errorbar(eplt,spes,xerr=np.ones(len(spes))*ewidth/1,yerr=spes_err,ls='',marker='.',markersize=5)
plt.xlabel("E$_{\gamma}$ (MeV)")
plt.ylabel("SPEs/gamma")

# 2. Determine the gamma spectrum from the source

In [ ]:
def read_hdf5_to_df(fname):
    """
    Reads an output HDF5 file from the nisource Geant4 simulation into a Pandas DataFrame.
    """
    
    # Open the file and get the ntuple.
    fn = h5py.File(fname,'r')
    ntuple = fn['default_ntuples']['nisource']
    
    # Fill a dictionary with the ntuple column values.
    df_values = {}
    for col in list(ntuple):

        # Only process H5 groups with actual data.
        if(isinstance(ntuple[col],h5py._hl.group.Group)):
            #print("Adding column",col,"...")
            if(ntuple[col]['pages'].dtype == object):
                df_values[col] = [x.decode('utf-8') for x in ntuple[col]['pages'][:]]
            else:
                df_values[col] = ntuple[col]['pages'][:]
             
    # Create the dataframe.
    df = pd.DataFrame.from_dict(df_values)
    
    # Add composite columns.
    ri = (df.xi**2 + df.yi**2 + df.zi**2)**0.5
    rf = (df.xf**2 + df.yf**2 + df.zf**2)**0.5
    Ki = (df.pxi**2 + df.pyi**2 + df.pzi**2 + df.mass**2)**0.5 - df.mass
    Kf = (df.pxf**2 + df.pyf**2 + df.pzf**2 + df.mass**2)**0.5 - df.mass
    
    df['ri'] = ri  # initial radius
    df['rf'] = rf  # final radius
    df['Ki'] = Ki  # initial kinetic energy
    df['Kf'] = Kf  # final kinetic energy
    
    # Compute time since initial decay.
    timin_evts = pd.DataFrame(df[df.ti > 0].groupby(['event'])['ti'].min())
    timin_evts = timin_evts.rename(columns={"ti": "ti_min"})
    df = pd.merge(df, timin_evts, on='event', how='outer')
    
    return df

In [ ]:
# Read in the results from HDF5.
df = read_hdf5_to_df('/data/jrenner/hk/standard/nicf_source_ntuple_100k_7cm.hdf5')
df.head()

In [ ]:
# Select gamma events that made it out of the source, with a time cut of 10 years.
Nevts = len(np.unique(df['event'].values))
years = (365*24*60*60*1.0e9)  # number of ns in a year
df_gammas_world = df[(df.particleName == 'gamma') & (df.volFinal == 'World')]
df_gammas_world_timecut = df_gammas_world[(df_gammas_world.ti < 10*years)]

In [ ]:
tcut1 = 0.5*years
tcut2 = 1*years
tcut3 = 10*years

fig = plt.figure(1)
fig.patch.set_alpha(0.0)
fig.set_figheight(4.0)
fig.set_figwidth(15.0)

# Without normalization
normed = False
ax1 = fig.add_subplot(121)
plt.hist(df_gammas_world.Kf,bins=100,density=normed,label="No time cut",color='red')
plt.hist(df_gammas_world[(df_gammas_world.ti < tcut3)].Kf,bins=100,density=normed,label="t < 10 years",color='green')
plt.hist(df_gammas_world[(df_gammas_world.ti < tcut2)].Kf,bins=100,density=normed,label="t < 1 year",color='blue')
plt.hist(df_gammas_world[(df_gammas_world.ti < tcut1)].Kf,bins=100,density=normed,label="t < 0.5 years",color='black')
plt.xlabel('Energy (MeV)')
plt.ylabel('Counts/bin')
plt.yscale("log")
plt.legend()

normed = True
ax2 = fig.add_subplot(122)
plt.hist(df_gammas_world[(df_gammas_world.ti < tcut3)].Kf,bins=100,density=normed,label="t < 10 years",color='green')
plt.hist(df_gammas_world[(df_gammas_world.ti < tcut2)].Kf,bins=100,density=normed,label="t < 1 year",color='blue')
plt.hist(df_gammas_world[(df_gammas_world.ti < tcut1)].Kf,bins=100,density=normed,label="t < 0.5 years",color='black')
plt.hist(df_gammas_world.Kf,bins=100,density=normed,label="No time cut",color='red')
plt.xlabel('Energy (MeV)')
plt.ylabel('Counts/bin')
plt.yscale("log")
plt.title("(Normalized)")
plt.legend()

In [ ]:
# Full gamma spectrum (no range restriction).
hfull, _, _ = plt.hist(df_gammas_world_timecut.Kf,bins=50,label="t < 10 years",color='green')
plt.xlabel('Energy (MeV)')
plt.ylabel('Counts/bin')
#plt.yscale('log')
plt.legend()
print("Total number of gammas:",np.sum(hfull))

In [ ]:
# Make the final gamma spectrum.
hgammas, hbins, _ = plt.hist(df_gammas_world_timecut.Kf,bins=50,label="t < 10 years",color='green',range=[0,10])
plt.xlabel('Energy (MeV)')
plt.ylabel('Counts/bin')
#plt.yscale('log')
plt.legend()
print("Total number of gammas:",np.sum(hgammas))

In [ ]:
# Multiply (gammas/decay/dE) * (SPEs/gamma) to get SPE spectrum.
hbin_centers = (hbins[:-1] + hbins[1:])/2
spe_spectrum = np.array(hgammas*spes/Nevts)
plt.bar(hbin_centers, spe_spectrum, align='center', width=0.2)
plt.xlabel('Energy (MeV)')
plt.ylabel('SPEs/decay/bin')
plt.yscale('log')
plt.title("Total SPEs/decay = {:.2e}".format(np.sum(spe_spectrum)))
print("Aligned bins:",(hbin_centers - eplt < 1e-6).all())

### PMT processing for all gammas

In [ ]:
plt.hist()

In [ ]:
# Compute key quantities for each PMT, including the mean number of photons per event.
N = len(df_photons)
pmt_mu, pmt_err, pmt_r, pmt_phi = [], [], [], []
print("Processing",N,"events...")
for npmt in range(2508):
    pmtID = npmt+1
    tag = 'pmt{}'.format(pmtID)
    print("tag is ",tag)
    x = df_pmt[df_pmt.pmt == pmtID]['x'].values[0]    
    y = df_pmt[df_pmt.pmt == pmtID]['y'].values[0]    
    z = df_pmt[df_pmt.pmt == pmtID]['z'].values[0]    
    
    pmt_mu.append(df_photons[tag].mean())
    pmt_err.append(df_photons[tag].std()/np.sqrt(N))
    pmt_r.append(np.sqrt(x**2 + y**2 + z**2))
    pmt_phi.append(np.arctan2(y,x))

In [ ]:
# Add the computed quantities to the PMT dataframe.
df_pmt['mu_photons'] = pmt_mu
df_pmt['err_photons'] = pmt_err
df_pmt['r'] = pmt_r
df_pmt['phi'] = pmt_phi

## Plots

In [ ]:
# A binned profile plot.
def profile_plot(X,Y,Yerr=None,nbins=10):
    
    # Use 1's for Yerr if none specified.
    if(Yerr is None):
        Yerr = np.ones(len(X))
    
    # Compute the bins.
    xmax = np.max(X)
    xmin = np.min(X)
    wbin = (xmax - xmin)/nbins
    xvals = np.arange(xmin,xmax,wbin) + wbin/2  # bin centers
    xerrs = np.ones(len(xvals))*wbin/2
    
    # Bin the values.
    nentries = np.zeros(nbins)
    yvals = np.zeros(nbins)
    yerrs = np.zeros(nbins)
    for xx,yy,yye in zip(X,Y,Yerr):
        nbin = int((xx - xmin)/wbin)
        if(nbin == nbins): nbin = nbins-1   # place xmax value in last bin
        yvals[nbin] += yy
        yerrs[nbin] += yye**2
        nentries[nbin] += 1
    
    # Normalize.
    yvals /= nentries
    yerrs = np.sqrt(yerrs)/nentries
    
    return xvals,xerrs,yvals,yerrs

In [ ]:
# Unbinned plot of mean number of events per PMT vs. PMT z.
fig = plt.figure()
fig.set_figheight(4.0)
fig.set_figwidth(16.0)
plt.plot(df_pmt['z'].values,df_pmt['mu_photons'].values,'.')
plt.xlabel("Z$_{\mathrm{PMT}}$ (cm)",fontsize=14)
plt.ylabel("SPE probability",fontsize=14)


In [ ]:
# Binned (profile) plot of mean number of events per PMT vs. PMT z.
fig = plt.figure()
fig.set_figheight(4.0)
fig.set_figwidth(16.0)
xvals,xerrs,yvals,yerrs = profile_plot(df_pmt['z'].values,df_pmt['mu_photons'].values,df_pmt['err_photons'].values,25)
plt.errorbar(xvals,yvals,xerr=xerrs,yerr=yerrs)
plt.xlabel("Z$_{\mathrm{PMT}}$ (cm)",fontsize=14)
plt.ylabel("SPE probability",fontsize=14)

In [ ]:
# An "event display"-style PMT plot.
#def pmts_plot(dfpmt, rxy=300, zlim=285, margin=20):
def pmts_plot(dfpmt, rxy=369.6, zlim=150, margin=20):
    
    ell_radius = 2
    x = dfpmt['x'].values
    y = dfpmt['y'].values
    z = dfpmt['z'].values
    spe = dfpmt['mu_photons'].values
    phi = np.arctan2(y,x) + np.pi
    
    cmap = matplotlib.cm.get_cmap('jet')
    
    spe_max = np.max(spe)
    spe_min = np.min(spe)
    #rxy = np.sqrt(x**2 + y**2)
    x_uw = rxy*phi
    
    fig = plt.figure()
    fig.set_figheight(18.0)
    fig.set_figwidth(21.0)
    
    ax_cbar = fig.add_subplot(3,12,10)
    cbar = matplotlib.colorbar.ColorbarBase(ax_cbar, cmap=cmap,
                       norm=matplotlib.colors.Normalize(vmin=spe_min, vmax=spe_max))
    
    # Plot the upper lid.
    x_ulid = x[z > zlim]
    y_ulid = y[z > zlim]
    spe_ulid = spe[z > zlim]
    ax_ulid = fig.add_subplot(3,3,2)
    ax_ulid.set_xlim(np.min(x_ulid)-margin,np.max(x_ulid)+margin)
    ax_ulid.set_ylim(np.min(y_ulid)-margin,np.max(y_ulid)+margin)
    ax_ulid.set_title("UPPER LID")
    ax_ulid.set_xlabel("x (cm)")
    ax_ulid.set_ylabel("y (cm)")
    for xplt,yplt,prob in zip(x_ulid,y_ulid,spe_ulid):
        col = cmap((prob - spe_min)/(spe_max - spe_min))
        ellipse = Ellipse((xplt, yplt), width=ell_radius * 2, height=ell_radius * 2, facecolor=col)
        ax_ulid.add_patch(ellipse)
    
    # Plot the cylinder
    xuw_cyl = x_uw[(z < zlim) & (z > -zlim)]
    z_cyl = z[(z < zlim) & (z > -zlim)]
    spe_cyl = spe[(z < zlim) & (z > -zlim)]
    ax_cyl = fig.add_subplot(3,1,2)
    ax_cyl.set_xlim(np.min(xuw_cyl)-margin,np.max(xuw_cyl)+margin)
    ax_cyl.set_ylim(np.min(z_cyl)-margin,np.max(z_cyl)+margin)
    ax_cyl.set_title("CYLINDER")
    ax_cyl.set_xlabel("distance along cylinder (cm)")
    ax_cyl.set_ylabel("z (cm)")
    for xplt,yplt,prob in zip(xuw_cyl,z_cyl,spe_cyl):
        col = cmap((prob - spe_min)/(spe_max - spe_min))
        ellipse = Ellipse((xplt, yplt), width=ell_radius * 2, height=ell_radius * 2, facecolor=col)
        ax_cyl.add_patch(ellipse)
        
    # Plot the lower lid.
    x_llid = x[z < -zlim]
    y_llid = y[z < -zlim]
    spe_llid = spe[z < -zlim]
    ax_llid = fig.add_subplot(3,3,8)
    ax_llid.set_xlim(np.min(x_llid)-margin,np.max(x_llid)+margin)
    ax_llid.set_ylim(np.min(y_llid)-margin,np.max(y_llid)+margin)
    ax_llid.set_title("LOWER LID")
    ax_llid.set_xlabel("x (cm)")
    ax_llid.set_ylabel("y (cm)")
    for xplt,yplt,prob in zip(x_llid,y_llid,spe_llid):
        col = cmap((prob - spe_min)/(spe_max - spe_min))
        ellipse = Ellipse((xplt, yplt), width=ell_radius * 2, height=ell_radius * 2, facecolor=col)
        ax_llid.add_patch(ellipse)

In [ ]:
pmts_plot(df_pmt)